In [1]:
#!/usr/bin/python

# import statements
import sys
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5 import QtWidgets, uic, QtCore, QtGui
from PyQt5.QtCore import QObject, pyqtSignal
import time
from datetime import datetime
import pygetwindow as gw

# for reading from an window
import cv2
from PIL import ImageGrab, Image
from numpy import asarray
import win32api
import winGuiAuto
import win32gui
import win32con
import numpy as np
import pywintypes

#For local CPU usage:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from mtcnn.mtcnn import MTCNN
from keras.models import load_model, Sequential, Model
from keras.layers import Dense, BatchNormalization, Dropout, Flatten
from keras.backend import clear_session, set_session
import tensorflow as tf

# VARIABLES
IPYTHON = True
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

INTERVAL_SECONDS = 3  ## Interval seconds between each time an emotion recognition output is produced
IMAGES_PER_INTERVAL = 6

LAYERS_TRAINABLE = True

Using TensorFlow backend.
c:\users\tobias\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\tobias\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\tobias\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [2]:
def getWindowNames():
    visibleWindows = []   
    
    #if gw.getWindowsWithTitle('Fotocamera'):
    #    visibleWindows.append('Fotocamera')
    
    for i in gw.getAllTitles():        
        notepadWindow = gw.getWindowsWithTitle(i)[0]
        
        if notepadWindow.isMinimized or notepadWindow.isMaximized:
            print(notepadWindow.title)
            visibleWindows.append(notepadWindow.title)
    
    return visibleWindows


print(getWindowNames())

app_prototype - Jupyter Notebook - Google Chrome
Fotocamera
React-Portfolio – Header.js IntelliJ IDEA
Bon Jovi - I'll Be There For You
app_prototype - Jupyter Notebook - Google Chrome
['app_prototype - Jupyter Notebook - Google Chrome', 'Fotocamera', 'React-Portfolio – Header.js IntelliJ IDEA', "Bon Jovi - I'll Be There For You", 'app_prototype - Jupyter Notebook - Google Chrome']


In [3]:
def custom_vgg_model():
    vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
    
    for layer in vgg_model.layers: 
        layer.trainable = LAYERS_TRAINABLE
        print(layer.name)
    
    last_layer = vgg_model.get_layer('pool5').output    
    x = Flatten(name='flatten')(last_layer)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    out1 = Dense(1, activation='tanh', name='out1')(x)
    out2 = Dense(1, activation='tanh', name='out2')(x)
    custom_vgg_model = Model(inputs= vgg_model.input, outputs= [out1, out2])
    
    return custom_vgg_model

model_top = custom_vgg_model()


input_1
conv1_1
conv1_2
pool1
conv2_1
conv2_2
pool2
conv3_1
conv3_2
conv3_3
pool3
conv4_1
conv4_2
conv4_3
pool4
conv5_1
conv5_2
conv5_3
pool5


In [4]:
## setting Keras sessions for each of the pretrained networks
sess = tf.Session()
graph = tf.get_default_graph()
set_session(sess)
detector = MTCNN()

## Second Network
# sess2 = tf.Session()
# graph = tf.get_default_graph()
# set_session(sess2)
# model_VGGFace = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Third Network
sess3 = tf.Session()
graph = tf.get_default_graph()
set_session(sess3)
model_top.load_weights("model_best.h5")

In [5]:
def detect_faces(image):
    global sess
    global graph
    with graph.as_default():
        set_session(sess)
        faces = detector.detect_faces(image)
        return np.array(faces)

In [6]:
def extract_face_from_image(image, required_size=(IMAGE_HEIGHT, IMAGE_WIDTH)):
    face = detect_faces(image) # content of face is a python dict

    if len(face) == 0:
        return []
    else:
        # extract the bounding box from the requested face
        box = np.asarray(face[0]['box'])
        box[box < 0] = 0
        x1, y1, width, height =  box

        x2, y2 = x1 + width, y1 + height
        # extract the face
        face_boundary = image[y1:y2, x1:x2]

        # resize pixels to the model size
        face_image = Image.fromarray(face_boundary)
        face_image = face_image.resize(required_size)
        face_array = asarray(face_image)
        return face_array

In [7]:
#def get_face_embedding(face):
#    samples = asarray(face, 'float32')
#    # prepare the data for the model
#    samples = preprocess_input(samples, version=2)
#    
#    global sess2
#    global graph
#    with graph.as_default():
#        set_session(sess2)
#        output = model_VGGFace.predict(samples)
#        return output

In [8]:
def return_prediction(path):
        #converting image to RGB color and save it
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

        #detect face in image, crop it then resize it
        face = extract_face_from_image(img)
        
        if face == []:
            return None, None
        else:
            print("Face detected")
            if face.ndim == 3:
                face = face.reshape((1, face.shape[0], face.shape[1], face.shape[2]))
            face = np.array(face)
            
            clear_session()
            global sess3
            global graph
            with graph.as_default():
                set_session(sess3)
                out1, out2 = model_top.predict(face) #make prediction and display the result
                val = out1[0][0] * 10
                ar = out2[0][0] * 10
                print("result: " + str(val) + ", " + str(ar))
                return val, ar    

## ProcessStream - Tasks:
         Task 1: Use Multithreading for GUI + Processing
         Task 2: Get pixels as an Input Stream (using OpenCV)
         https://theailearner.com/2018/10/16/recording-a-specific-window-using-opencv-python/
         Task 3: Preprocessing & Landmark detection
         Task 4: Read in Python Model

In [10]:
class ProcessStream(QThread):   
    output = pyqtSignal(object)
    def __init__(self):
        super(ProcessStream, self).__init__()
        self.active = True

    def run(self):
        # Task 1: Use Multithreading ???   for GUI + Input Stream
        # Task 2: Get pixels as an Input Stream (using OpenCV)
        # Task 3: Preprocessing & Landmark detection
        # Task 4: Read in Python Model
        image = ImageGrab.grab()
        height,width,channel = np.array(image).shape

        out = cv2.VideoWriter('video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, (width,height))
        
        val_list = []
        arr_list = []
        i = 0
        
        while self.active == True:
            # image = ImageGrab.grab(rect)
            image = ImageGrab.grab()
            out.write(cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB))
            
            imgUMat = np.float32(image)
            cv2.imwrite("test.jpg", imgUMat)

            val, arr = return_prediction("test.jpg")
            if val != None:
                val_list.append(val)
                arr_list.append(arr)
                if len(val_list) > int(ER_SECONDS / SNAP_SECONDS):
                    val_list.pop(0)
                    arr_list.pop(0)
                
                i = i + 1
                if i % IM == 0:
                    valence = sum(val_list) / len(val_list)
                    arousal = sum(arr_list) / len(arr_list)
                    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    result = str(now) + "   Valence: " + str(round(valence, 4)) + "\n" + str(now) + "   Arousal: " + str(round(arousal, 4))
                    self.output.emit(result)
                
            time.sleep(int(INTERVAL_SECONDS/IMAGES_PER_INTERVAL))
        out.release()
        
    def stop(self):
        print("STOP Thread")
        self.active = False

In [11]:
#class CreateContent(threading.Thread):
#    def dynamicContentCreation():
        # Last step in Prototype
        # Speech Recognition ( Speech -> to -> Text )
        # Make us of a 'faked' chat-bot (decision tree)
#        return 0

In [ ]:
# Create GUI with PyQt
selected_window = ""

class Ui_MainWindow(object): 
    def setupUi(self, window): 
        super().__init__()
        self.window = window
        #self.window.box_selection.addItems(inputList)
        self.window.btn_start.clicked.connect(self.getSelection)
        self.recording = False
        self.totalResults = ""
        #self.thread1 = ProcessStream(self.window.box_selection, window)
        self.thread1 = ProcessStream()
        self.thread1.output.connect(self.addResults)
        app.aboutToQuit.connect(self.closeEvent)
        
    def getSelection(self): 
        if self.recording == False:
            #selected_window = str(self.window.box_selection.currentText())
            #print("Analytics got started! Selected window: " + selected_window)
            print("Analytics got started!")
            self.recording = True
            # changing the text of label after button got clicked 
            self.window.btn_start.setText("Stop Analytics")
            self.thread1.start() # This actually causes the thread to run
        else:
            self.recording = False
            self.thread1.stop()
                        
            self.window.btn_start.setText("Start Recording")
            # self.thread1 = ProcessStream(self.window.box_selection, self.window)  # recreate thread
            self.thread1 = ProcessStream()
            self.thread1.output.connect(self.addResults)
    
    def closeEvent(self):
        print('Close button pressed')
        self.recording = False
        self.thread1.stop()
        
        if IPYTHON:
            app.deleteLater
        else:
            sys.exit(0)
    
    def addResults(self, inputText):
        self.totalResults = (inputText + "\n" + self.totalResults)
        self.window.box_results.setText(self.totalResults)


if __name__ == "__main__":  
    app = QtWidgets.QApplication(sys.argv)  
    window = uic.loadUi("dialog_2.ui")
    
    myApp = Ui_MainWindow()  
    # myApp.setupUi(window, getWindowNames())
    myApp.setupUi(window)  
      
      
    if IPYTHON == False:
        window.show() 
        sys.exit(app.exec_())
    else:
        window.show()
        app.exec_()

Analytics got started!



c:\users\tobias\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


Face detected
result: 0.5875082686543465, 1.9619350135326385
Face detected
result: -0.20952671766281128, 1.7156150937080383
Face detected
result: -0.10765597224235535, 1.688503473997116
Face detected
result: -0.1770811714231968, 2.027571201324463
Face detected
result: -0.10601568967103958, 2.239316701889038
Face detected
result: -1.6132524609565735, 3.465612232685089
Face detected
result: 0.11138797737658024, 1.4457005262374878
Face detected
result: 0.031198605429381132, 1.6897067427635193
Face detected
result: 0.22774465382099152, 2.2624383866786957
Face detected
result: -0.23171847686171532, 1.9432294368743896
Face detected
result: -0.04243884701281786, 1.7992478609085083
Face detected
result: -0.1907404512166977, 2.5669029355049133
Face detected
result: 0.27493100613355637, 1.3767020404338837
Face detected
result: -0.17132019624114037, 1.7614257335662842
Face detected
result: 0.03389077028259635, 1.9700779020786285
Face detected
result: -0.09554079733788967, 2.0438985526561737
Face 